# Finding non-dissonant bassizans in In Nomine corpus

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import pandas as pd
import re
import requests
import os
import glob as glob
import verovio

We start by importing a piece and looking for the durational ratio and the melodic motion of the cantizans.

In [2]:
prefix = 'Music_Files/' 

# Add your filename here
mei_file = 'Tallis-1.xml'

# These join the strings and import the piece
url = prefix + mei_file
piece = importScore(url)

#print(piece.metadata)

In [3]:
def convertTuple(tup):
    str = '_'.join(tup)
    return str

# Find combined melodic and durational ngrams of n=2
# Offsets must be set to 'last'

n = 2
dr = piece.durationalRatios().round(3)
mel = piece.melodic(kind='d')
ng = piece.ngrams(df=mel, other=dr, n=n, offsets='last')

ngs = ng.stack().apply(convertTuple)
out1 = ngs.to_frame()

out1.reset_index(level=[0,1])

syncope = '-2_0.5_2_(\d+(?:\.\d+)?)'

results_sync = ngs[ngs.str.match(syncope)].to_frame().index.get_level_values(0)
results_sync.to_list()

har = piece.harmonic(kind="d", compound=False)
har_mel_ng = piece.ngrams(df=har, other=mel, n=n, offsets='last')
har_mel_hits = har_mel_ng.stack()
out2 = har_mel_hits.to_frame()

out2.reset_index(level=[0,1])

harm_cad = '3_-5, (\d+?)'
results_har_mel = har_mel_hits[har_mel_hits.str.match(harm_cad)].to_frame().index.get_level_values(0)
results_har_mel.to_list()

harm_cad2 = '3_4, (\d+?)'
results_har_mel2 = har_mel_hits[har_mel_hits.str.match(harm_cad2)].to_frame().index.get_level_values(0)
results_har_mel2.to_list()

complete_results_har_mel = results_har_mel.append(results_har_mel2)
complete_results_har_mel.sort_values()

shared_entries = list(set(results_sync).intersection(complete_results_har_mel))
shared_entries

syncopes_table = ng.loc[results_sync]

modules_table = har_mel_ng.loc[complete_results_har_mel]
modules_table.sort_index()

complete_table = pd.merge(modules_table, syncopes_table, left_index=True, right_index=True)

complete_stack = complete_table.stack().apply(convertTuple)
c1 = complete_stack.to_frame()

harm_cad_tuple = '3___-_5_,_ _(\d+?)'
harm_cad_tuple2 = '3___4_,_ _(\d+?)'

mod_5d = complete_stack[complete_stack.str.match(harm_cad_tuple)]
mod_df = mod_5d.to_frame().reset_index(level=[1])

mod_df.columns = ['voices', 'mod']
mod_df.drop('mod', axis=1, inplace=True)

mod_4u = complete_stack[complete_stack.str.match(harm_cad_tuple2)]
mod4_df = mod_4u.to_frame().reset_index(level=[1])

mod4_df.columns = ['voices', 'mod']
mod4_df.drop('mod', axis=1, inplace=True)

mod_df = mod_df[~mod_df.index.duplicated(keep='first')]
mod4_df = mod4_df[~mod4_df.index.duplicated(keep='first')]

temp_mod = pd.concat([mod_df, mod4_df], axis=1)
temp_mod.columns = ['voices', 'voices4']
temp_mod['voice_combo'] = temp_mod['voices'].fillna(mod4_df['voices'])
temp_mod = temp_mod[['voice_combo']]

c1_cant = complete_stack[complete_stack.str.match(syncope)]
c1_cant_df = c1_cant.to_frame().reset_index(level=[1])
c1_cant_df.columns = ['cantizans', 'syncope']
c1_cant_df = c1_cant_df[~c1_cant_df.reset_index().duplicated().values]
c1_cant_df.drop('syncope', axis=1, inplace=True)
c1_cant_stack = c1_cant_df.stack().to_frame()
c1_cant_stack = c1_cant_stack.reset_index(level=[1])

duple_check_df = pd.concat([c1_cant_stack, temp_mod], axis=1)
duple_check_df.columns = ['cvf', 'voice', 'voice_combo']
duple_check_done = duple_check_df[~duple_check_df.apply(lambda x: x.voice not in x.voice_combo, axis=1)]

complete_table_cant_voice = duple_check_done[['voice', 'voice_combo']]
complete_table_cant_voice.columns = ['cantizans', 'voice_combo']

complete_table_cant_voice["voice_combo"] = complete_table_cant_voice.apply(lambda x: x["voice_combo"].replace(x["cantizans"], "").strip(), axis=1)
complete_table_cant_voice['voice_combo'] = complete_table_cant_voice['voice_combo'].str.replace('_','')
complete_table_cant_voice.columns = ['cantizans_voice', 'bassizans_voice']

#df of all notes in piece
notes_df = piece.notes()

#find shared offsets
notes_df = notes_df[notes_df.index.isin(complete_table_cant_voice.index)]
notes_df_new = pd.concat([complete_table_cant_voice, notes_df], axis=1)

one = notes_df_new['[I]']
two = notes_df_new['[II]']
three = notes_df_new['[III]']
four = notes_df_new['[IV]']

notes_df_new['cantizans_pitch1'] = np.where(notes_df_new['cantizans_voice']
                       == '[I]', one,'' )
notes_df_new['cantizans_pitch2'] = np.where(notes_df_new['cantizans_voice']
                       == '[II]', two,'' )
notes_df_new['cantizans_pitch3'] = np.where(notes_df_new['cantizans_voice']
                       == '[III]', three,'' )
notes_df_new['cantizans_pitch4'] = np.where(notes_df_new['cantizans_voice']
                       == '[IV]', four,'' )

notes_df_new["cantizans_pitch"] = notes_df_new["cantizans_pitch1"] + notes_df_new["cantizans_pitch2"] + notes_df_new["cantizans_pitch3"] + notes_df_new["cantizans_pitch4"]

b_one = notes_df_new['[I]']
b_two = notes_df_new['[II]']
b_three = notes_df_new['[III]']
b_four = notes_df_new['[IV]']

notes_df_new['bassizans_pitch1'] = np.where(notes_df_new['bassizans_voice']
                       == '[I]', b_one,'' )
notes_df_new['bassizans_pitch2'] = np.where(notes_df_new['bassizans_voice']
                       == '[II]', b_two,'' )
notes_df_new['bassizans_pitch3'] = np.where(notes_df_new['bassizans_voice']
                       == '[III]', b_three,'' )
notes_df_new['bassizans_pitch4'] = np.where(notes_df_new['bassizans_voice']
                       == '[IV]', b_four,'' )

notes_df_new["bassizans_pitch"] = notes_df_new["bassizans_pitch1"] + notes_df_new["bassizans_pitch2"] + notes_df_new["bassizans_pitch3"] + notes_df_new["bassizans_pitch4"]

penultimate = notes_df_new[notes_df_new.columns[~notes_df_new.columns.isin(['[I]', '[II]', '[III]', '[IV]', 'cantizans_pitch1', 'cantizans_pitch2', 'cantizans_pitch3', 'cantizans_pitch4', 'bassizans_pitch1', 'bassizans_pitch2', 'bassizans_pitch3', 'bassizans_pitch4'])]]
penultimate = penultimate[['cantizans_voice', 'cantizans_pitch', 'bassizans_voice', 'bassizans_pitch']]

final_table_mb = piece.detailIndex(penultimate, offset=True)
final_table_mb

C:\Users\eribe952\AppData\Local\Temp\ipykernel_17420\1469603772.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_table_cant_voice["voice_combo"] = complete_table_cant_voice.apply(lambda x: x["voice_combo"].replace(x["cantizans"], "").strip(), axis=1)
C:\Users\eribe952\AppData\Local\Temp\ipykernel_17420\1469603772.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_table_cant_voice['voice_combo'] = complete_table_cant_voice['voice_combo'].str.replace('_','')


,,,cantizans_voice,cantizans_pitch,bassizans_voice,bassizans_pitch
Measure,Beat,Offset,,,,
51,1.0,400.0,[I],D5,[III],D3
55,1.0,432.0,[III],G4,[IV],G3
60,1.0,472.0,[I],G4,[IV],G2


In [4]:
measure_tb = final_table_mb.reset_index(level=[0,1,2])
measure = measure_tb['Measure'].to_list()

for x in measure:
    minus = x - 1
    piece.verovioPrintExample(minus, x)

Score:
File Name:  Tallis-1
Thomas Tallis
In Nomine I
Measures: 50-51


Score:
File Name:  Tallis-1
Thomas Tallis
In Nomine I
Measures: 54-55


Score:
File Name:  Tallis-1
Thomas Tallis
In Nomine I
Measures: 59-60
